In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

W0529 17:19:31.450861 139962895238976 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to /home/ajay/tensorflow_datasets/mnist/1.0.0. Subsequent calls will reuse this data.


In [3]:
BUFFER_SIZE = 10 # Use a much larger value for real code.
BATCH_SIZE = 64
NUM_EPOCHS = 5

In [4]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [5]:
train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).take(5)
test_data = mnist_test.map(scale).batch(BATCH_SIZE).take(5)

STEPS_PER_EPOCH = 5

train_data = train_data.take(STEPS_PER_EPOCH)
test_data = test_data.take(STEPS_PER_EPOCH)

In [12]:
image_batch, label_batch = next(iter(train_data))
print(image_batch.shape)

(64, 28, 28, 1)


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(train_data, epochs=NUM_EPOCHS)

Epoch 1/5
5/5 [==============================] - 1s 238ms/step - loss: 1.6273 - accuracy: 0.4656
Epoch 2/5
5/5 [==============================] - 0s 43ms/step - loss: 0.6980 - accuracy: 0.8000
Epoch 3/5
5/5 [==============================] - 0s 51ms/step - loss: 0.4440 - accuracy: 0.8969
Epoch 4/5
5/5 [==============================] - 0s 41ms/step - loss: 0.4090 - accuracy: 0.9125
Epoch 5/5
5/5 [==============================] - 0s 48ms/step - loss: 0.3085 - accuracy: 0.9375


In [11]:
loss, acc = model.evaluate(test_data)
print("Loss {}, Accuracy {}".format(loss, acc))

      5/Unknown - 0s 48ms/step - loss: 1.5194 - accuracy: 0.6687Loss 1.5193892002105713, Accuracy 0.668749988079071
